# GCN on ogbn-proteins

from https://github.com/snap-stanford/ogb/blob/master/examples/nodeproppred/proteins/gnn.py by Matthias Fey

In [1]:
import argparse

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

In [2]:
class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [3]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(
            GCNConv(in_channels, hidden_channels, normalize=False))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, normalize=False))
        self.convs.append(
            GCNConv(hidden_channels, out_channels, normalize=False))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

In [4]:
parser = argparse.ArgumentParser(description='OGBN-Proteins (GNN)')
parser.add_argument('--device', type=int, default=0)
parser.add_argument('--log_steps', type=int, default=1)
parser.add_argument('--use_sage', action='store_true', default=True)
parser.add_argument('--num_layers', type=int, default=3)
parser.add_argument('--hidden_channels', type=int, default=256)
parser.add_argument('--dropout', type=float, default=0.0)
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epochs', type=int, default=1000)
parser.add_argument('--eval_steps', type=int, default=5)
parser.add_argument('--runs', type=int, default=10)
args = parser.parse_args(args=[])
print(args)

device = f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

dataset = PygNodePropPredDataset(
    name='ogbn-proteins', transform=T.ToSparseTensor(attr='edge_attr'))
data = dataset[0]

# Move edge features to node features.
data.x = data.adj_t.mean(dim=1)
data.adj_t.set_value_(None)

split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

# GCN Model
model = GCN(data.num_features, args.hidden_channels, 112,
                args.num_layers, args.dropout).to(device)

# Pre-compute GCN normalization.
adj_t = data.adj_t.set_diag()
deg = adj_t.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj_t = deg_inv_sqrt.view(-1, 1) * adj_t * deg_inv_sqrt.view(1, -1)
data.adj_t = adj_t

data = data.to(device)

evaluator = Evaluator(name='ogbn-proteins')
logger = Logger(args.runs, args)

Namespace(device=0, dropout=0.0, epochs=1000, eval_steps=5, hidden_channels=256, log_steps=1, lr=0.01, num_layers=3, runs=10, use_sage=True)


In [5]:
def train(model, data, train_idx, optimizer):
    model.train()
    criterion = torch.nn.BCEWithLogitsLoss()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = criterion(out, data.y[train_idx].to(torch.float))
    loss.backward()
    optimizer.step()

    return loss.item()

In [6]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    y_pred = model(data.x, data.adj_t)

    train_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['rocauc']
    valid_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['rocauc']
    test_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['rocauc']

    return train_rocauc, valid_rocauc, test_rocauc

In [7]:
model

GCN(
  (convs): ModuleList(
    (0): GCNConv(8, 256)
    (1): GCNConv(256, 256)
    (2): GCNConv(256, 112)
  )
)

In [8]:
for run in range(args.runs):
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    for epoch in range(1, 1 + args.epochs):
        loss = train(model, data, train_idx, optimizer)

        if epoch % args.eval_steps == 0:
            result = test(model, data, split_idx, evaluator)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_rocauc, valid_rocauc, test_rocauc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_rocauc:.2f}%, '
                      f'Valid: {100 * valid_rocauc:.2f}% '
                      f'Test: {100 * test_rocauc:.2f}%')

    logger.print_statistics(run)
logger.print_statistics()

Run: 01, Epoch: 05, Loss: 0.4565, Train: 34.89%, Valid: 34.31% Test: 37.46%
Run: 01, Epoch: 10, Loss: 0.4265, Train: 36.78%, Valid: 34.93% Test: 37.57%
Run: 01, Epoch: 15, Loss: 0.4324, Train: 36.79%, Valid: 33.42% Test: 36.07%
Run: 01, Epoch: 20, Loss: 0.4165, Train: 37.90%, Valid: 33.87% Test: 36.10%
Run: 01, Epoch: 25, Loss: 0.4051, Train: 40.64%, Valid: 34.65% Test: 36.15%
Run: 01, Epoch: 30, Loss: 0.3986, Train: 41.42%, Valid: 34.30% Test: 35.47%
Run: 01, Epoch: 35, Loss: 0.3925, Train: 44.17%, Valid: 35.02% Test: 35.36%
Run: 01, Epoch: 40, Loss: 0.3876, Train: 46.00%, Valid: 35.89% Test: 35.54%
Run: 01, Epoch: 45, Loss: 0.3830, Train: 48.53%, Valid: 37.28% Test: 36.04%
Run: 01, Epoch: 50, Loss: 0.3791, Train: 50.23%, Valid: 38.65% Test: 36.66%
Run: 01, Epoch: 55, Loss: 0.3756, Train: 51.50%, Valid: 40.38% Test: 37.39%
Run: 01, Epoch: 60, Loss: 0.3719, Train: 52.98%, Valid: 42.38% Test: 38.16%
Run: 01, Epoch: 65, Loss: 0.3682, Train: 54.36%, Valid: 44.51% Test: 39.13%
Run: 01, Epo

Run: 01, Epoch: 540, Loss: 0.2902, Train: 78.59%, Valid: 76.52% Test: 67.65%
Run: 01, Epoch: 545, Loss: 0.2898, Train: 78.59%, Valid: 76.46% Test: 67.67%
Run: 01, Epoch: 550, Loss: 0.2895, Train: 78.66%, Valid: 76.44% Test: 67.77%
Run: 01, Epoch: 555, Loss: 0.2892, Train: 78.78%, Valid: 76.51% Test: 67.93%
Run: 01, Epoch: 560, Loss: 0.2889, Train: 78.89%, Valid: 76.62% Test: 68.15%
Run: 01, Epoch: 565, Loss: 0.2886, Train: 78.94%, Valid: 76.60% Test: 68.07%
Run: 01, Epoch: 570, Loss: 0.2906, Train: 78.81%, Valid: 76.54% Test: 68.75%
Run: 01, Epoch: 575, Loss: 0.2888, Train: 78.89%, Valid: 76.63% Test: 68.80%
Run: 01, Epoch: 580, Loss: 0.2890, Train: 79.10%, Valid: 76.77% Test: 68.49%
Run: 01, Epoch: 585, Loss: 0.2882, Train: 79.09%, Valid: 76.55% Test: 68.28%
Run: 01, Epoch: 590, Loss: 0.2873, Train: 79.31%, Valid: 76.92% Test: 68.96%
Run: 01, Epoch: 595, Loss: 0.2873, Train: 79.39%, Valid: 76.99% Test: 68.98%
Run: 01, Epoch: 600, Loss: 0.2866, Train: 79.41%, Valid: 76.89% Test: 68.78%

Run: 02, Epoch: 70, Loss: 0.3643, Train: 55.21%, Valid: 45.30% Test: 39.60%
Run: 02, Epoch: 75, Loss: 0.3614, Train: 56.00%, Valid: 46.76% Test: 40.35%
Run: 02, Epoch: 80, Loss: 0.3587, Train: 56.75%, Valid: 48.01% Test: 40.96%
Run: 02, Epoch: 85, Loss: 0.3561, Train: 57.94%, Valid: 49.73% Test: 41.83%
Run: 02, Epoch: 90, Loss: 0.3533, Train: 58.84%, Valid: 51.00% Test: 42.53%
Run: 02, Epoch: 95, Loss: 0.3505, Train: 60.01%, Valid: 52.21% Test: 43.02%
Run: 02, Epoch: 100, Loss: 0.3478, Train: 61.35%, Valid: 53.74% Test: 44.12%
Run: 02, Epoch: 105, Loss: 0.3452, Train: 62.19%, Valid: 54.76% Test: 44.82%
Run: 02, Epoch: 110, Loss: 0.3428, Train: 63.10%, Valid: 55.96% Test: 45.83%
Run: 02, Epoch: 115, Loss: 0.3406, Train: 64.49%, Valid: 57.92% Test: 47.34%
Run: 02, Epoch: 120, Loss: 0.3386, Train: 65.18%, Valid: 59.02% Test: 48.35%
Run: 02, Epoch: 125, Loss: 0.3370, Train: 65.90%, Valid: 60.30% Test: 49.44%
Run: 02, Epoch: 130, Loss: 0.3356, Train: 66.65%, Valid: 61.70% Test: 50.60%
Run: 

Run: 02, Epoch: 605, Loss: 0.2899, Train: 79.78%, Valid: 77.36% Test: 70.07%
Run: 02, Epoch: 610, Loss: 0.2873, Train: 79.07%, Valid: 76.80% Test: 69.91%
Run: 02, Epoch: 615, Loss: 0.2850, Train: 80.21%, Valid: 77.47% Test: 70.02%
Run: 02, Epoch: 620, Loss: 0.2850, Train: 79.64%, Valid: 77.17% Test: 70.08%
Run: 02, Epoch: 625, Loss: 0.2855, Train: 80.17%, Valid: 77.55% Test: 70.56%
Run: 02, Epoch: 630, Loss: 0.2840, Train: 80.07%, Valid: 77.40% Test: 70.00%
Run: 02, Epoch: 635, Loss: 0.2843, Train: 80.32%, Valid: 77.66% Test: 70.50%
Run: 02, Epoch: 640, Loss: 0.2835, Train: 80.04%, Valid: 77.48% Test: 70.52%
Run: 02, Epoch: 645, Loss: 0.2834, Train: 80.18%, Valid: 77.61% Test: 70.67%
Run: 02, Epoch: 650, Loss: 0.2830, Train: 80.29%, Valid: 77.66% Test: 70.69%
Run: 02, Epoch: 655, Loss: 0.2827, Train: 80.30%, Valid: 77.65% Test: 70.71%
Run: 02, Epoch: 660, Loss: 0.2822, Train: 80.44%, Valid: 77.75% Test: 70.84%
Run: 02, Epoch: 665, Loss: 0.2820, Train: 80.52%, Valid: 77.77% Test: 70.72%

Run: 03, Epoch: 135, Loss: 0.3321, Train: 65.96%, Valid: 61.96% Test: 50.55%
Run: 03, Epoch: 140, Loss: 0.3307, Train: 67.89%, Valid: 65.75% Test: 53.45%
Run: 03, Epoch: 145, Loss: 0.3292, Train: 67.05%, Valid: 64.16% Test: 52.71%
Run: 03, Epoch: 150, Loss: 0.3279, Train: 68.43%, Valid: 66.77% Test: 54.81%
Run: 03, Epoch: 155, Loss: 0.3268, Train: 68.18%, Valid: 66.27% Test: 54.58%
Run: 03, Epoch: 160, Loss: 0.3258, Train: 68.41%, Valid: 66.68% Test: 55.08%
Run: 03, Epoch: 165, Loss: 0.3246, Train: 69.08%, Valid: 67.86% Test: 56.14%
Run: 03, Epoch: 170, Loss: 0.3238, Train: 69.37%, Valid: 68.30% Test: 56.63%
Run: 03, Epoch: 175, Loss: 0.3230, Train: 69.59%, Valid: 68.59% Test: 56.93%
Run: 03, Epoch: 180, Loss: 0.3221, Train: 69.78%, Valid: 68.79% Test: 57.18%
Run: 03, Epoch: 185, Loss: 0.3217, Train: 69.28%, Valid: 67.81% Test: 56.30%
Run: 03, Epoch: 190, Loss: 0.3224, Train: 70.48%, Valid: 69.78% Test: 58.19%
Run: 03, Epoch: 195, Loss: 0.3206, Train: 70.22%, Valid: 69.31% Test: 57.85%

...

Run: 08, Epoch: 460, Loss: 0.3026, Train: 77.63%, Valid: 75.35% Test: 66.28%
Run: 08, Epoch: 465, Loss: 0.2971, Train: 77.40%, Valid: 74.92% Test: 66.15%
Run: 08, Epoch: 470, Loss: 0.2979, Train: 77.28%, Valid: 74.92% Test: 65.96%
Run: 08, Epoch: 475, Loss: 0.2972, Train: 76.92%, Valid: 74.48% Test: 65.35%
Run: 08, Epoch: 480, Loss: 0.2967, Train: 77.11%, Valid: 74.67% Test: 65.70%
Run: 08, Epoch: 485, Loss: 0.2957, Train: 77.37%, Valid: 74.89% Test: 65.93%
Run: 08, Epoch: 490, Loss: 0.2947, Train: 77.64%, Valid: 75.11% Test: 66.33%
Run: 08, Epoch: 495, Loss: 0.2941, Train: 77.68%, Valid: 75.14% Test: 66.31%
Run: 08, Epoch: 500, Loss: 0.2938, Train: 77.67%, Valid: 75.08% Test: 66.43%
Run: 08, Epoch: 505, Loss: 0.2933, Train: 77.68%, Valid: 75.08% Test: 66.34%
Run: 08, Epoch: 510, Loss: 0.2929, Train: 77.86%, Valid: 75.24% Test: 66.64%
Run: 08, Epoch: 515, Loss: 0.2946, Train: 78.48%, Valid: 75.76% Test: 67.17%
Run: 08, Epoch: 520, Loss: 0.2926, Train: 78.40%, Valid: 75.70% Test: 67.08%

Run: 08, Epoch: 995, Loss: 0.2727, Train: 82.72%, Valid: 79.05% Test: 71.75%
Run: 08, Epoch: 1000, Loss: 0.2722, Train: 82.78%, Valid: 79.11% Test: 71.87%
Run 08:
Highest Train: 82.78
Highest Valid: 79.11
  Final Train: 82.78
   Final Test: 71.87
Run: 09, Epoch: 05, Loss: 0.4475, Train: 34.62%, Valid: 33.85% Test: 37.11%
Run: 09, Epoch: 10, Loss: 0.4231, Train: 36.75%, Valid: 34.62% Test: 37.17%
Run: 09, Epoch: 15, Loss: 0.4291, Train: 36.98%, Valid: 33.61% Test: 36.02%
Run: 09, Epoch: 20, Loss: 0.4158, Train: 38.18%, Valid: 33.61% Test: 35.71%
Run: 09, Epoch: 25, Loss: 0.4048, Train: 41.33%, Valid: 34.40% Test: 35.73%
Run: 09, Epoch: 30, Loss: 0.3973, Train: 42.58%, Valid: 34.20% Test: 35.11%
Run: 09, Epoch: 35, Loss: 0.3899, Train: 45.43%, Valid: 35.06% Test: 35.24%
Run: 09, Epoch: 40, Loss: 0.3842, Train: 48.09%, Valid: 36.45% Test: 35.64%
Run: 09, Epoch: 45, Loss: 0.3797, Train: 50.33%, Valid: 38.28% Test: 36.31%
Run: 09, Epoch: 50, Loss: 0.3760, Train: 52.32%, Valid: 40.24% Test: 

Run: 09, Epoch: 525, Loss: 0.2882, Train: 79.13%, Valid: 76.39% Test: 67.88%
Run: 09, Epoch: 530, Loss: 0.2885, Train: 78.91%, Valid: 76.25% Test: 68.09%
Run: 09, Epoch: 535, Loss: 0.2872, Train: 79.15%, Valid: 76.39% Test: 68.21%
Run: 09, Epoch: 540, Loss: 0.2888, Train: 79.64%, Valid: 76.73% Test: 68.07%
Run: 09, Epoch: 545, Loss: 0.2884, Train: 79.82%, Valid: 76.83% Test: 68.37%
Run: 09, Epoch: 550, Loss: 0.2870, Train: 79.80%, Valid: 76.78% Test: 68.36%
Run: 09, Epoch: 555, Loss: 0.2860, Train: 79.74%, Valid: 76.79% Test: 68.30%
Run: 09, Epoch: 560, Loss: 0.2861, Train: 79.60%, Valid: 76.72% Test: 68.45%
Run: 09, Epoch: 565, Loss: 0.2861, Train: 79.43%, Valid: 76.65% Test: 68.65%
Run: 09, Epoch: 570, Loss: 0.2852, Train: 79.85%, Valid: 76.89% Test: 68.70%
Run: 09, Epoch: 575, Loss: 0.2857, Train: 80.11%, Valid: 77.04% Test: 68.63%
Run: 09, Epoch: 580, Loss: 0.2852, Train: 79.75%, Valid: 76.83% Test: 68.77%
Run: 09, Epoch: 585, Loss: 0.2846, Train: 79.75%, Valid: 76.86% Test: 68.89%

Run: 10, Epoch: 55, Loss: 0.3747, Train: 52.51%, Valid: 40.74% Test: 37.39%
Run: 10, Epoch: 60, Loss: 0.3707, Train: 54.11%, Valid: 43.47% Test: 38.70%
Run: 10, Epoch: 65, Loss: 0.3669, Train: 54.99%, Valid: 45.42% Test: 39.49%
Run: 10, Epoch: 70, Loss: 0.3636, Train: 55.52%, Valid: 46.59% Test: 39.94%
Run: 10, Epoch: 75, Loss: 0.3606, Train: 55.94%, Valid: 47.76% Test: 40.49%
Run: 10, Epoch: 80, Loss: 0.3577, Train: 57.25%, Valid: 49.70% Test: 41.64%
Run: 10, Epoch: 85, Loss: 0.3544, Train: 58.43%, Valid: 51.19% Test: 42.55%
Run: 10, Epoch: 90, Loss: 0.3509, Train: 59.84%, Valid: 52.79% Test: 43.76%
Run: 10, Epoch: 95, Loss: 0.3480, Train: 61.16%, Valid: 54.13% Test: 44.49%
Run: 10, Epoch: 100, Loss: 0.3443, Train: 63.30%, Valid: 56.84% Test: 46.40%
Run: 10, Epoch: 105, Loss: 0.3417, Train: 64.49%, Valid: 58.53% Test: 47.54%
Run: 10, Epoch: 110, Loss: 0.3393, Train: 65.27%, Valid: 59.83% Test: 48.63%
Run: 10, Epoch: 115, Loss: 0.3370, Train: 65.95%, Valid: 61.26% Test: 50.01%
Run: 10,

Run: 10, Epoch: 590, Loss: 0.2879, Train: 79.26%, Valid: 76.81% Test: 68.76%
Run: 10, Epoch: 595, Loss: 0.2877, Train: 79.22%, Valid: 76.73% Test: 68.90%
Run: 10, Epoch: 600, Loss: 0.2872, Train: 79.26%, Valid: 76.75% Test: 68.93%
Run: 10, Epoch: 605, Loss: 0.2889, Train: 79.54%, Valid: 77.02% Test: 68.85%
Run: 10, Epoch: 610, Loss: 0.2871, Train: 79.60%, Valid: 77.04% Test: 69.02%
Run: 10, Epoch: 615, Loss: 0.2873, Train: 79.61%, Valid: 77.09% Test: 68.99%
Run: 10, Epoch: 620, Loss: 0.2877, Train: 79.59%, Valid: 77.03% Test: 69.06%
Run: 10, Epoch: 625, Loss: 0.2867, Train: 79.56%, Valid: 76.90% Test: 69.01%
Run: 10, Epoch: 630, Loss: 0.2861, Train: 79.60%, Valid: 76.95% Test: 69.11%
Run: 10, Epoch: 635, Loss: 0.2860, Train: 79.71%, Valid: 77.05% Test: 69.25%
Run: 10, Epoch: 640, Loss: 0.2855, Train: 79.81%, Valid: 77.13% Test: 69.40%
Run: 10, Epoch: 645, Loss: 0.2861, Train: 79.55%, Valid: 76.80% Test: 69.35%
Run: 10, Epoch: 650, Loss: 0.2853, Train: 79.86%, Valid: 77.14% Test: 69.38%